In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import seaborn as sns

from scipy.stats import pearsonr

sns.set_style("darkgrid")
np.random.seed(930525)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 200)

warnings.simplefilter('once')

%matplotlib inline
%load_ext watermark
%watermark --iversions

pandas  1.1.4
numpy   1.19.5
seaborn 0.10.1



In [2]:
sra_assembly_summary = "../data/assembly_summary.sra.csv"
sra_500_subset = "../data/srastudy.500.txt"
bac_gtdbtk_results = "/mnt/btrfs/data/type_1/genomes_assembly_wgs_500_out/classify/gtdbtk.bac120.summary.tsv"
arc_gtdbtk_results = "/mnt/btrfs/data/type_1/genomes_assembly_wgs_500_out/classify/gtdbtk.ar122.summary.tsv"

/home/bhillmann/.conda/envs/type_1/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
df_sra = pd.read_csv(sra_assembly_summary)
sras_subset = pd.read_csv(sra_500_subset, header=None)
sras_subset.columns = ["Run"]

df_sra = pd.merge(df_sra, sras_subset, on="Run", how="inner")

In [4]:
dfs = []

for file in (bac_gtdbtk_results, arc_gtdbtk_results):
    df = pd.read_csv(file, sep="\t")
    dfs.append(df)

df_gtdbtk = pd.concat(dfs)

In [5]:
df_gtdbtk["# assembly_accession"] = ["_".join(_.split("_")[:2]) for _ in df_gtdbtk["user_genome"]]

In [6]:
df_merged = pd.merge(df_sra, df_gtdbtk, how="left", on = "# assembly_accession")

In [7]:
df_merged_in_gtdb = df_merged.loc[df_merged["in_gtdb"] | [type(_) == type("") for _ in df_merged["closest_placement_reference"]]].copy()

In [8]:
df_merged_in_gtdb["closest_assembly_accession"] = ""

df_merged_in_gtdb.loc[~df_merged_in_gtdb["in_gtdb"], "closest_assembly_accession"] = df_merged_in_gtdb.loc[~df_merged_in_gtdb["in_gtdb"], "closest_placement_reference"]
df_merged_in_gtdb.loc[df_merged_in_gtdb["in_gtdb"], "closest_assembly_accession"] = df_merged_in_gtdb.loc[df_merged_in_gtdb["in_gtdb"], "# assembly_accession"]

In [9]:
df_merged_in_gtdb.loc[:, ["# assembly_accession", "closest_assembly_accession"]]

df_merged_in_gtdb.to_csv("../data/assembly_accessions_500.csv")

In [10]:
df_merged_in_gtdb

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,...,closest_placement_af,pplacer_taxonomy,classification_method,note,"other_related_references(genome_id,species_name,radius,ANI,AF)",aa_percent,translation_table,red_value,warnings,closest_assembly_accession
0,GCF_900112205.1,PRJNA224116,SAMN05444422,FOKW00000000.1,representative genome,148448,148448,Halobiforma haloterrestris,strain=DSM 13078,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCF_900112205.1
1,GCF_009729515.1,PRJNA224116,SAMN09933021,WGGD00000000.1,representative genome,523847,47303,Sulfuracidifex metallicus DSM 6482 = JCM 9184,strain=DSM 6482,NaN,...,1.00,d__Archaea;p__Thermoproteota;c__Thermoproteia;...,taxonomic classification defined by topology a...,topological placement and ANI have congruent s...,"GCF_001316085.1, s__Sulfolobus_B sp001316085, ...",95.67,11.0,NaN,NaN,GCF_001316045.1
2,GCF_900113245.1,PRJNA224116,SAMN04488063,FOOQ00000000.1,representative genome,553467,553467,Halopelagius inordinatus,strain=CGMCC 1.7739,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCF_900113245.1
3,GCF_900112175.1,PRJNA224116,SAMN04487947,FOYT00000000.1,representative genome,553469,553469,Halogeometricum rufum,strain=CGMCC 1.7736,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCF_900112175.1
4,GCF_900215575.1,PRJNA224116,SAMN06269185,OBEJ00000000.1,representative genome,558529,558529,Natronoarchaeum philippinense,strain=DSM 27208,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCF_900215575.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,GCF_005871185.1,PRJNA224116,SAMN11445132,SWKN00000000.1,representative genome,2570352,2570352,Limnobacter alexandrii,strain=LZ-4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCF_005871185.1
494,GCF_005780165.1,PRJNA224116,SAMN11569541,VAUV00000000.1,representative genome,2576071,2576071,Phragmitibacter flavus,strain=MG-N-17,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCF_005780165.1
497,GCF_011682065.1,PRJNA224116,SAMN12688462,VUYU00000000.1,representative genome,2607910,2607910,Massilia rubra,strain=CCM 8692,NaN,...,0.77,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,taxonomic classification defined by topology a...,NaN,"GCF_003953935.1, s__Massilia sp003953935, 95.0...",96.37,11.0,0.99533,Genome not assigned to closest species as it f...,GCF_002752675.1
498,GCF_902499555.1,PRJNA224116,SAMEA5787750,CABVMM000000000.1,representative genome,2687242,2687242,Mesonia oceanica,strain=ISS653,NaN,...,1.00,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,taxonomic classification defined by topology a...,topological placement and ANI have congruent s...,"GCF_000423405.1, s__Mesonia mobilis, 95.0, 81....",98.47,11.0,NaN,NaN,GCA_002413625.1
